In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import sys
import logging
import os

rootFolder=os.getcwd()+"/"
baseFolder=rootFolder+"temp/"   
chromeProfilePath=baseFolder+'ChromeProfile'
chromeWebDriverPath = '/usr/lib/chromium-browser/chromedriver'
refreshUrl='https://onlineplus.mofidonline.com/Home/Default/page-1'

options = webdriver.ChromeOptions()
# if not (showUI):
#     options.add_argument('headless')
#     options.add_argument('window-size=1920x1080')
#     options.add_argument("disable-gpu")
options.add_argument("--log-level=3");
options.add_argument("user-data-dir="+chromeProfilePath) #Path to your chrome profile
driver = webdriver.Chrome(executable_path=chromeWebDriverPath, options=options)
driver.get(refreshUrl)
driver

<selenium.webdriver.chrome.webdriver.WebDriver (session="7b580f57b1fa9a2a1fe13a8a88da2c82")>

In [9]:
def getMofidAccountData():
    import tinydb
    securitySettingFilename=rootFolder+'securitySetting.json'
    username="TypeUserNameIn:"+securitySettingFilename
    password="TypePasswordIn:"+securitySettingFilename
    mofidAccountTable="mofidAccount"
    try:
        db = tinydb.TinyDB(securitySettingFilename)
        userTable = db.table(mofidAccountTable)
        user = userTable.all()
        username=user[0]["Username"]
        password=user[0]["Password"]
        db.close()
    except:
        print("Error in username and password Easy Trader.")
        db.purge_table(mofidAccountTable)
        userTable = db.table(mofidAccountTable)
        userTable.insert({"Username":username, "Password":password})
        db.close()
    return {"username":username, "password":password}
def checkIsLogin():
    #driver.switch_to.window(driver.window_handles[0])
    #<a class="signout " href="/Account/Logout">
    inputStr="//a[@class='signout ' and @href='/Account/Logout']"
    signOutElems = driver.find_elements_by_xpath(inputStr) 
    if (signOutElems):
        return True
    return False
def refresh():
    driver.switch_to.window(driver.window_handles[0])
    driver.refresh()
    #logger.info(str(datetime.datetime.now())+" Refreshed..")

In [10]:
def loginWithMofidAccount():
    import time
    while(True):
        refresh()
        time.sleep(2)
        if (checkIsLogin()):
            break 
        driver.get(refreshUrl)
        elem = driver.find_element_by_class_name("mofid__input-button")
        elem.click()

        counter=0
        mofidLoginWindowExist=False
        while(True):
            if (len(driver.window_handles)==2):
                mofidLoginWindowExist=True
                break
            else:
                time.sleep(1)
                counter+=1
                if counter>10:break
        if (mofidLoginWindowExist):
            driver.switch_to.window(driver.window_handles[1])
            username=driver.find_element_by_id("Username")
            password=driver.find_element_by_id("Password")
            loginButton=driver.find_element_by_id("submit_btn")
            mofidAccount=getMofidAccountData()
            username.send_keys(mofidAccount["username"])
            password.send_keys(mofidAccount["password"])
            loginButton.click()
        driver.switch_to.window(driver.window_handles[0])
        if (checkIsLogin()):
            break
        time.sleep(10)
        if (checkIsLogin()):
            break    
        time.sleep(45)
        if (checkIsLogin()):
            break   
    driver.switch_to_default_content()    
loginWithMofidAccount()    

NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=80.0.3987.149)


In [12]:

driver.window_handles

['CDwindow-455F496E7F5BE0402322F475952A6333']